# Word Search Solver
I was inspired by [this Reddit post](https://www.reddit.com/r/puzzles/comments/1e5ml2e/my_wife_says_word_searches_are_too_easy_so_i_made/) to write an automated [word search](https://en.wikipedia.org/wiki/Word_search) solver. It's a great excuse to work with OCR and image processing tools.

I thought I'd need some image segmenting and clusering tools to determine the grid, but it turns out for nicely formatted grids given as a nice digital image we can actually do things much more easily with just numpy.

## Solution

### Tools

In [ ]:
from PIL import Image, ImageDraw

In [ ]:
import numpy as np

In [ ]:
import pytesseract

### Preprocessing / Transcription

In [ ]:
def axis_gridding(ax):
    streak_indices = np.argwhere(np.diff(ax)).ravel() # excluding start and end
    streak_lengths = np.diff(streak_indices)
    streak_centers = streak_indices[:-1] + streak_lengths/2
    char_centers = streak_centers[0::2]
    n = len(char_centers)
    grid_spacing, grid_offset = np.polyfit(np.arange(n), char_centers, 1)
    return n, grid_spacing, grid_offset

def transcribe_wordsearch(img, verbose=False):
    img = img.convert('L')
    arr = np.array(img)
    bin_arr = arr < 128
    
    nx, dx, ox = axis_gridding(np.any(bin_arr, axis=0))
    ny, dy, oy = axis_gridding(np.any(bin_arr, axis=1))
    if verbose:
        print(f"{nx} columns, spaced {dx}, starting at {ox}")
        print(f"{ny} rows, spaced {dy}, starting at {oy}")

    grid = []
    for i in range(ny):
        grid.append([])
        y0 = max(0, oy + i * dy - dy//2)
        y1 = min(img.size[1], oy + i * dy + dy//2)
        for j in range(nx):
            x0 = max(0, ox + j * dx - dx//2)
            x1 = min(img.size[0], ox + j * dx + dx//2)
            cimg = img.crop((x0, y0, x1, y1))
            c = pytesseract.image_to_string(cimg, config='--psm 10').strip()
            grid[-1].append(c)
            if verbose:
                print(f"Processed row {i} / {ny}, column {j} / {nx}: {c}")
    return grid, (nx, dx, ox), (ny, dy, oy)

In [ ]:
def check_grid(grid_img, nx, dx, ox, ny, dy, oy):
    check_grid_img = grid_img.copy()
    draw_check_grid_img = ImageDraw.Draw(check_grid_img)
    for i in range(nx):
        draw_check_grid_img.line((ox+dx*i,0,ox+dx*i,check_grid_img.size[1]), fill=(255,0,0), width=1)
    for i in range(nx):
        draw_check_grid_img.line((0,oy+dy*i,check_grid_img.size[0],oy+dy*i), fill=(255,0,0), width=1)
    return check_grid_img

### Search Algorithm

In [ ]:
def build_trie(wordlist):
    trie = dict()
    for word in wordlist:
        curr = trie
        for c in word.upper():
            if c not in curr:
                curr[c] = dict()
            curr = curr[c]
        if 'END' not in curr:
            curr['END'] = word
    return trie

In [ ]:
def wordsearch(grid, wordlist, wrap=False, backwards=True):
    trie = build_trie(wordlist)
    nx,ny = len(grid),len(grid[0])
    result = dict()
    for i in range(nx):
        for j in range(ny):
            for di in [-1,0,1]:
                dj_options = [-1,0,1] if backwards else [0,1]
                for dj in dj_options:
                    if di == 0 and dj == 0:
                        continue
                    ip,jp = i,j
                    curr = trie
                    c = grid[i][j]
                    while c in curr:
                        curr = curr[c]
                        if 'END' in curr:
                            w = curr['END']
                            if w not in result:
                                result[w] = set()
                            if w != w[::-1] or ((ip, jp), (-di,-dj)) not in result[w]: # filter palindromes
                                result[w].add(((i,j), (di,dj)))
                        ip += di
                        jp += dj
                        if not ((0 <= ip < len(grid)) and (0 <= jp < len(grid[0]))):
                            if wrap:
                                ip %= len(grid)
                                jp %= len(grid)
                            else:
                                break
                        c = grid[ip][jp]
    return result

### Display Results

In [ ]:
def draw_solution(img, solution, nx, dx, ox, ny, dy, oy, circling_size=0.8, width=3, color=(255,0,0)):
    solved = img.copy()
    draw_solved = ImageDraw.Draw(solved)

    rx, ry = dx/2*circling_size, dy/2*circling_size
    for word,locations in solution.items():
        for ((iy0,ix0),(idy,idx)) in locations:
            angle = np.arctan2(idy, idx) % (2 * np.pi)
            cdx, cdy = np.cos(angle-np.pi/2)*rx, np.sin(angle-np.pi/2) * ry
            
            deg = angle*180/np.pi
            x0, y0 = ox + dx * ix0, oy + dy * iy0
            draw_solved.arc(((x0-rx,y0-ry),(x0+rx,y0+ry)), deg+90, deg+90+180, fill=color, width=width)
            ix1,iy1 = (ix0+idx*(len(word)-1))%nx, (iy0+idy*(len(word)-1))%ny
            x1, y1 = ox+dx*ix1, oy+dy*iy1
            draw_solved.arc(((x1-rx,y1-ry),(x1+rx,y1+ry)), deg-90, deg+90, fill=color, width=width)
            
            ipx,ipy = ix0,iy0
            ix,iy = ix0,iy0
            for i,c in enumerate(word):
                if not ((0 <= ix+idx < nx) and (0 <= iy+idy < ny)) or i == len(word)-1:
                    px,py = ox + dx * ipx, oy + dy * ipy
                    x,y = ox + dx * ix, oy + dy * iy
                    draw_solved.line(((px-cdx,py-cdy),(x-cdx,y-cdy)), fill=color, width=width)
                    draw_solved.line(((px+cdx,py+cdy),(x+cdx,y+cdy)), fill=color, width=width)
                    ipx,ipy = (ix+idx)%nx, (iy+idy)%ny
                ix = (ix+idx)%nx
                iy = (iy+idy)%ny
    return solved

## Example 1: u/xuol's from Reddit
Original post [here](https://www.reddit.com/r/puzzles/comments/1e5ml2e/my_wife_says_word_searches_are_too_easy_so_i_made/).

### Open Data

In [ ]:
import requests
import io

In [ ]:
puzzle_img = Image.open(io.BytesIO(requests.get('https://i.redd.it/fg9pqcl7v3dd1.png').content))
puzzle_img

In [ ]:
grid_img = puzzle_img.crop((0,0,puzzle_img.size[0]-200,puzzle_img.size[1]))
grid_img.resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
wordlist_img = puzzle_img.crop((puzzle_img.size[0]-200,0,puzzle_img.size[0],puzzle_img.size[1]))
wordlist_img.resize((wordlist_img.size[0]//3, wordlist_img.size[1]//3))

In [ ]:
wordlist = pytesseract.image_to_string(wordlist_img).split()
wordlist

### Apply Solution

#### Transcribe

In [ ]:
grid, (nx, dx, ox), (ny, dy, oy) = transcribe_wordsearch(grid_img, verbose=True)

In [ ]:
check_grid(grid_img, nx, dx, ox, ny, dy, oy).resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
print('\n'.join(' '.join(row) for row in grid))

#### Correct Transcription

In [ ]:
remap = {'|': 'I', 'l': 'I'}
fixedgrid = [[remap.get(c, c) for c in row] for row in grid]
print('\n'.join(' '.join(row) for row in fixedgrid))

#### Perform Search

In [ ]:
solution = wordsearch(fixedgrid, wordlist)
solution

In [ ]:
for w in wordlist:
    if w not in solution:
        print(f"- '{w}' NOT FOUND")
    else:
        print(f"- '{w}' found {len(solution[w])} times")

### Visualize Solution

In [ ]:
solution_img = draw_solution(grid_img, solution, nx, dx, ox, ny, dy, oy, circling_size=0.6)
solution_img.save('uxol-solution.png')
solution_img

## Example 2: u/OTDisLanguish's from Reddit
Original post [here](https://www.reddit.com/r/puzzles/comments/1e62t9s/worst_search_blame_uxuol/).

In [ ]:
puzzle_img = Image.open(io.BytesIO(requests.get('https://i.redd.it/8u47ucjmg7dd1.png').content))
puzzle_img

In [ ]:
grid_img = puzzle_img.crop((0,0,puzzle_img.size[0]-620,puzzle_img.size[1]))
grid_img.resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
wordlist_img = puzzle_img.crop((puzzle_img.size[0]-620,166,puzzle_img.size[0],puzzle_img.size[1]))
wordlist_img.resize((wordlist_img.size[0]//3, wordlist_img.size[1]//3))

In [ ]:
wordlist = pytesseract.image_to_string(wordlist_img).split()
wordlist

In [ ]:
grid, (nx, dx, ox), (ny, dy, oy) = transcribe_wordsearch(grid_img, verbose=True)

In [ ]:
check_grid(grid_img, nx, dx, ox, ny, dy, oy).resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
print('\n'.join(' '.join(row) for row in grid))

In [ ]:
remap = {'Ss': 'S'}
fixedgrid = [[remap.get(c, c) for c in row] for row in grid]
print('\n'.join(' '.join(row) for row in fixedgrid))

In [ ]:
solution = wordsearch(fixedgrid, wordlist, wrap=True)
solution

In [ ]:
for w in wordlist:
    if w not in solution:
        print(f"- '{w}' NOT FOUND")
    else:
        print(f"- '{w}' found {len(solution[w])} times")

In [ ]:
solution_img = draw_solution(grid_img, solution, nx, dx, ox, ny, dy, oy, circling_size=0.9)
solution_img.save('OTDisLanguish-solution.png')
solution_img

## Example 3: u/jaykamilav5's from Reddit
Original post [here](https://www.reddit.com/r/puzzles/comments/1e6e25s/i_made_a_playable_online_version_of_uxuols/).

In [ ]:
puzzle_img = Image.open(io.BytesIO(requests.get('https://i.redd.it/af5xjlt7oadd1.png').content))
puzzle_img

In [ ]:
grid_img = puzzle_img.crop((0,200,puzzle_img.size[0],puzzle_img.size[1]))
grid_img.resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
wordlist_img = puzzle_img.crop((0,0,puzzle_img.size[0],200))
wordlist_img.resize((wordlist_img.size[0]//3, wordlist_img.size[1]//3))

In [ ]:
wordlist = pytesseract.image_to_string(wordlist_img).split()
wordlist

In [ ]:
grid, (nx, dx, ox), (ny, dy, oy) = transcribe_wordsearch(grid_img, verbose=True)

In [ ]:
check_grid(grid_img, nx, dx, ox, ny, dy, oy).resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
print('\n'.join(' '.join(row) for row in grid))

In [ ]:
remap = {'Ss': 'S'}
fixedgrid = [[remap.get(c, c) for c in row] for row in grid]
print('\n'.join(' '.join(row) for row in fixedgrid))

In [ ]:
solution = wordsearch(fixedgrid, wordlist, backwards=True)
solution

In [ ]:
for w in wordlist:
    if w not in solution:
        print(f"- '{w}' NOT FOUND")
    else:
        print(f"- '{w}' found {len(solution[w])} times")

In [ ]:
solution_img = draw_solution(grid_img, solution, nx, dx, ox, ny, dy, oy, circling_size=0.7)
solution_img.save('jaykamilav5-solution.png')
solution_img

## Example 4: u/armbargain's from Reddit
Original post [here](https://www.reddit.com/r/puzzles/comments/1e64032/this_abomination_is_titled_strain/).

In [ ]:
puzzle_img = Image.open(io.BytesIO(requests.get('https://i.redd.it/7n2fvpmtt7dd1.jpeg').content))
puzzle_img

In [ ]:
grid_img = puzzle_img.crop((0,0,puzzle_img.size[0],puzzle_img.size[1]-300))
grid_img.resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
wordlist_img = puzzle_img.crop((0,puzzle_img.size[1]-300,puzzle_img.size[0],puzzle_img.size[1]))
wordlist_img.resize((wordlist_img.size[0]//3, wordlist_img.size[1]//3))

In [ ]:
wordlist = pytesseract.image_to_string(wordlist_img).split()
wordlist

In [ ]:
grid, (nx, dx, ox), (ny, dy, oy) = transcribe_wordsearch(grid_img, verbose=True)

In [ ]:
check_grid(grid_img, nx, dx, ox, ny, dy, oy).resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
print('\n'.join(' '.join(row) for row in grid))

In [ ]:
remap = {'Ss': 'S', 'i':'I', 'li':'I', 'TT':'T', 'if':'I', 'il':'I'}
fixedgrid = [[remap.get(c, c) for c in row] for row in grid]
print('\n'.join(' '.join(row) for row in fixedgrid))

In [ ]:
solution = wordsearch(fixedgrid, wordlist, backwards=True)
solution

In [ ]:
for w in wordlist:
    if w not in solution:
        print(f"- '{w}' NOT FOUND")
    else:
        print(f"- '{w}' found {len(solution[w])} times")

In [ ]:
solution_img = draw_solution(grid_img, solution, nx, dx, ox, ny, dy, oy, circling_size=0.7)
solution_img.save('armbargain-solution.png')
solution_img

### Example 5: u/armbargain's 2nd from Reddit
Original post [here](https://www.reddit.com/r/puzzles/comments/1e5u0vr/inspired_by_uxuol_i_made_this_quickly/).

In [ ]:
puzzle_img = Image.open(io.BytesIO(requests.get('https://i.redd.it/40pre8lud5dd1.jpeg').content))
puzzle_img

In [ ]:
grid_img = puzzle_img.crop((140,42,puzzle_img.size[0]-160,puzzle_img.size[1]-470))
grid_img.resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
wordlist_img = puzzle_img.crop((0,puzzle_img.size[1]-400,puzzle_img.size[0],puzzle_img.size[1]))
wordlist_img.resize((wordlist_img.size[0]//3, wordlist_img.size[1]//3))

In [ ]:
wordlist = pytesseract.image_to_string(wordlist_img).split()
wordlist

In [ ]:
grid, (nx, dx, ox), (ny, dy, oy) = transcribe_wordsearch(grid_img, verbose=True)

In [ ]:
check_grid(grid_img, nx, dx, ox, ny, dy, oy).resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
print('\n'.join(' '.join(row) for row in grid))

In [ ]:
remap = {'Cc': 'C', '(@':'C', '7':'T', '¢':'C'}
fixedgrid = [[remap.get(c, c) for c in row] for row in grid]
print('\n'.join(' '.join(row) for row in fixedgrid))

In [ ]:
solution = wordsearch(fixedgrid, wordlist, backwards=True)
solution

In [ ]:
for w in wordlist:
    if w not in solution:
        print(f"- '{w}' NOT FOUND")
    else:
        print(f"- '{w}' found {len(solution[w])} times")

In [ ]:
solution_img = draw_solution(grid_img, solution, nx, dx, ox, ny, dy, oy, circling_size=1)
solution_img.save('armbargain-solution.png')
solution_img

## References
- https://pypi.org/project/pytesseract/
- https://en.wikipedia.org/wiki/Word_search
- https://scikit-image.org/docs/stable/user_guide/tutorial_segmentation.html
- https://scikit-image.org/docs/0.20.x/auto_examples/segmentation/plot_regionprops.html